# Create the regionalized version of EF 3.1 in brightway

In [1]:
import bw2data as bd
import numpy as np
import pandas as pd
from mescal import *

In [2]:
bd.projects.set_current(f'ecoinvent3.9.1')

In [3]:
ef_cfs = pd.read_excel('lcia/EF-LCIAMethod_CF(EF-v3.1).xlsx', sheet_name='lciamethods_CF')

## Create the new methods

In [4]:
biosphere_names_exceptions = {
    'nitrogen monoxide': 'nitric oxide',
    'nitrous oxide': 'dinitrogen monoxide',
    'HFC-134a': 'Ethane, 1,1,1,2-tetrafluoro-, HFC-134a',
}

In [5]:
def ecoinvent_compatible_categories(name, cat0, cat1, cat2):
    
    # if name in biosphere_names_exceptions.keys():
    #     name = biosphere_names_exceptions[name]
    
    if cat0 == 'Emissions':
        cat1 = cat1.replace('Emissions to ', '')
        cat2 = cat2.replace('Emissions to ', '')
        cat2 = cat2.replace('and', '+')
        
        if cat2 == 'air, unspecified':
            cat2 = None
        elif cat2 == 'air, unspecified (long-term)':
            cat2 = 'low population density, long-term'
            
        if '(biogenic)' in name:
            name = name.replace(' (biogenic)', ', non-fossil')
        elif '(fossil)' in name:
            name = name.replace(' (fossil)', ', fossil')
        elif '(land use change)' in name:
            name = name.replace(' (land use change)', ', to soil or biomass stock')
        
        if 'Particles' in name:
            name.replace('Particles', 'Particulate Matter, ')
            
            if '(PM10)' in name:
                name = name.replace('(PM10)', '> 2.5 um and < 10um')
            elif '(PM0.2 - PM2.5)' in name:
                name = name.replace('(PM0.2 - PM2.5)', '< 2.5 um')
        
        return name, cat1, cat2
    
    elif cat0 == 'Land use':
        cat1 = cat1.replace('Land ', '')
        name = f'{cat1}, {name}'
        cat1 = 'natural resource'
        cat2 = 'land'
        return name, cat1, cat2
    
    elif cat0 == 'Resources':
        cat2 = f"in {cat1.replace('Resources from ', '')}"
        cat1 = 'natural resource'
        return name, cat1, cat2
    
    else:
        raise ValueError(f"Category0 {cat0} not recognized")

In [6]:
biosphere_db_spatialized = Database('biosphere3_spatialized_flows')
biosphere_db = Database('biosphere3')

Getting activity data


100%|██████████| 62222/62222 [00:00<00:00, 127954.66it/s]


Adding exchange data to activities


0it [00:00, ?it/s]


Filling out exchange data


...

Adding exchange data to activities


0it [00:00, ?it/s]


Filling out exchange data


100%|██████████| 4718/4718 [00:00<?, ?it/s]

Loaded biosphere3 from brightway!


In [38]:
def create_cf_list(impact_category, ef_cfs=ef_cfs):
    
    regionalized_cfs = []
    unmapped = []
    df = ef_cfs[ef_cfs['LCIAMethod_name'] == impact_category].reset_index(drop=True)
    
    for i in range(len(df)):
        
        flow_name = df.loc[i, 'FLOW_name']
        
        flow_location = df.loc[i, 'LCIAMethod_location']
        
        flow_cf = df.loc[i, 'CF EF3.1']
        
        if flow_location is np.nan:
            db_name = 'biosphere3'
        else:
            flow_name = f'{flow_name}, {flow_location}'
            db_name = 'biosphere3_spatialized_flows'
        
        category0 = df.loc[i, 'FLOW_class0']
        category1 = df.loc[i, 'FLOW_class1']
        category2 = df.loc[i, 'FLOW_class2']
        flow_name, category1, category2 = ecoinvent_compatible_categories(flow_name, category0, category1, category2)
        
        flow_name = flow_name[0].upper() + flow_name[1:] # adding a capital letter to the flow name
        
        if category2 is not None:
            try:
                flow_code = [i for i in bd.Database(db_name).search(flow_name, limit=1000) if 
                         (i['name'] == flow_name)
                         & (i['categories'] == (category1, category2))
                         ][0]['code']
            except IndexError:
                unmapped.append((db_name, flow_name, (category1, category2)))
                continue
        
        else:
            try:
                flow_code = [i for i in bd.Database(db_name).search(flow_name, limit=1000) if 
                         (i['name'] == flow_name)
                         & (i['categories'] == (category1,))
                         ][0]['code']
            except IndexError:
                unmapped.append((db_name, flow_name, (category1, category2)))
                continue
        
        regionalized_cfs.append(((db_name, flow_code), flow_cf))
    
    return regionalized_cfs, unmapped

In [8]:
ef_units = {
    'acidification': 'mol H+-Eq',
    'climate change': 'kg CO2-Eq',
    'climate change: biogenic': 'kg CO2-Eq',
    'climate change: fossil': 'kg CO2-Eq',
    'climate change: land use and land use change': 'kg CO2-Eq',
    'ecotoxicity: freshwater': 'CTUe',
    'ecotoxicity: freshwater, inorganics': 'CTUe',
    'ecotoxicity: freshwater, organics': 'CTUe',
    'energy resources: non-renewable': 'MJ, net calorific value',
    'eutrophication: freshwater': 'kg P-Eq',
    'eutrophication: marine': 'kg N-Eq',
    'eutrophication: terrestrial': 'mol N-Eq',
    'human toxicity: carcinogenic': 'CTUh',
    'human toxicity: carcinogenic, inorganics': 'CTUh',
    'human toxicity: carcinogenic, organics': 'CTUh',
    'human toxicity: non-carcinogenic': 'CTUh',
    'human toxicity: non-carcinogenic, inorganics': 'CTUh',
    'human toxicity: non-carcinogenic, organics': 'CTUh',
    'ionising radiation: human health': 'kBq U235-Eq',
    'land use': 'dimensionless',
    'material resources: metals/minerals': 'kg Sb-Eq',
    'ozone depletion': 'kg CFC-11-Eq',
    'particulate matter formation': 'disease incidence',
    'photochemical oxidant formation: human health': 'kg NMVOC-Eq',
    'water use': 'm3 world eq. deprived',
}

In [39]:
write = False
impact_category_list = list(ef_cfs.LCIAMethod_name.unique())[0:3]
ef_regionalized_cfs = {}
not_mapped = {}
for cat in impact_category_list:
    print(cat)
    new_ef_regionalized_cfs, new_unmapped = create_cf_list(cat)
    ef_regionalized_cfs[cat] = new_ef_regionalized_cfs
    not_mapped[cat] = new_unmapped
    if write:
        ef_regionalized_method = bd.Method(('EF v3.1 regionalized', cat.lower()))
        ef_regionalized_method_metadata = {'unit': ef_units[cat.lower()]}
        ef_regionalized_method.register(**ef_regionalized_method_metadata)
        ef_regionalized_method.write(new_ef_regionalized_cfs)

Acidification
Climate change
Climate change-Biogenic


## Validation

Check that all global flows were correctly reproduced

In [57]:
missing_flows = []
for cat in impact_category_list:
    cat_bw = cat.replace('-', ': ')
    method = bd.Method([i for i in bd.methods if i[0] == 'EF v3.1' and i[1] == cat_bw.lower()][0])
    cfs = method.load()
    for i in cfs:
        if i not in ef_regionalized_cfs[cat]:
            # print(i)
            i_name = [j for j in biosphere_db.db_as_list if j['code'] == i[0][1]][0]['name']
            missing_flows.append(i_name)

In [53]:
pd.DataFrame(data=ef_cfs.FLOW_name.unique(), columns=['Name']).to_csv('ef31_names.csv', index=False)

In [60]:
pd.DataFrame(data=list(set(missing_flows)), columns=['Name']).to_csv('biosphere3_names.csv', index=False)